In [1]:
import pandas as pd
import itertools

origins = ['A', 'B', 'C']
destinations = ['A', 'B', 'C']
dias_llegada = list(range(1, 8))  # 1 (Lundi) à 7 (Dimanche)

combinations = list(itertools.product(origins, destinations, dias_llegada))
combinations = [combo for combo in combinations if combo[0] != combo[1]]

df = pd.DataFrame(combinations, columns=['Id_origin', 'Id_destination', 'Dias_llegada'])
df['Reward'] = 1

# Condition if monday 
condition = (
    (df['Dias_llegada'] == 1) &  # Lundi
    (
        ((df['Id_origin'] == 'A') & (df['Id_destination'] == 'B')) |
        ((df['Id_origin'] == 'B') & (df['Id_destination'] == 'A'))
    )
)

df.loc[condition, 'Reward'] = -1

print(df)

df.to_csv('donnees_synthetiques.csv', index=False)

   Id_origin Id_destination  Dias_llegada  Reward
0          A              B             1      -1
1          A              B             2       1
2          A              B             3       1
3          A              B             4       1
4          A              B             5       1
5          A              B             6       1
6          A              B             7       1
7          A              C             1       1
8          A              C             2       1
9          A              C             3       1
10         A              C             4       1
11         A              C             5       1
12         A              C             6       1
13         A              C             7       1
14         B              A             1      -1
15         B              A             2       1
16         B              A             3       1
17         B              A             4       1
18         B              A             5       1


Truck 1 cant go from A<=>B on monday and sunday 

In [2]:
import pandas as pd
import random
from datetime import datetime, timedelta

num_trucks = 13  # Number of trucks
num_days = 7    # Number of days in the week
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']  # Days of the week
cities = ['A', 'B', 'C', 'D', 'E', 'F']  # List of cities

data = []

for truck in range(1, num_trucks + 1):
    for day in range(num_days):
        date = datetime.now() + timedelta(days=day)  # Calculate the date for each day
        
        city_origin = random.choice(cities)  # Select a random origin city
        city_destination = random.choice(cities)  # Select a random destination city
        
        # Ensure that city_origin and city_destination are not the same
        if city_origin == city_destination:
            continue  # Skip this entry if they are the same

        # Apply the constraint for Truck 1
        if truck == 1 and (days_of_week[day] == 'Monday' or days_of_week[day] == 'Sunday'):
            if (city_origin == 'A' and city_destination == 'B') or (city_origin == 'B' and city_destination == 'A'):
                continue  # Skip this entry if the constraint is violated

        # Append the data for the current truck and day
        data.append({
            'Truck': f'Truck_{truck}',  # Truck identifier
            'Day': days_of_week[day],  # Day of the week
            'City_Origin': city_origin,  # Origin city
            'City_Destination': city_destination  # Destination city
        })

# Create a DataFrame from the generated data
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('dataset_truck_orders_cities_updated.csv', index=False)

print("Dataset saved as 'dataset_truck_orders_cities_updated.csv'")

Dataset saved as 'dataset_truck_orders_cities_updated.csv'


Any truck cant go from A<=>B on monday and sunday 

In [3]:
import pandas as pd
import random
from datetime import datetime, timedelta

num_trucks = 10  # Number of trucks
num_days = 30    # Number of days to generate data for a whole month
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']  # Days of the week
cities = ['A', 'B', 'C', 'D', 'E', 'F']  # List of cities

data = []

for truck in range(1, num_trucks + 1):
    for day in range(num_days):
        date = datetime.now() + timedelta(days=day)  # Calculate the date for each day
        
        city_origin = random.choice(cities)  # Select a random origin city
        city_destination = random.choice(cities)  # Select a random destination city
        
        # Ensure that city_origin and city_destination are not the same
        if city_origin == city_destination:
            continue  # Skip this entry if they are the same

        # Apply the new constraint: no truck can go from A to B or B to A on Mondays and Sundays
        if (days_of_week[day % 7] == 'Monday' or days_of_week[day % 7] == 'Sunday'):
            if (city_origin == 'A' and city_destination == 'B') or (city_origin == 'B' and city_destination == 'A'):
                continue  # Skip this entry if the constraint is violated

        # Append the data for the current truck and day
        data.append({
            'Truck': f'Truck_{truck}',  # Truck identifier
            'Day': days_of_week[day % 7],  # Day of the week
            'City_Origin': city_origin,  # Origin city
            'City_Destination': city_destination  # Destination city
        })

# Create a DataFrame from the generated data
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('dataset_truck_orders_cities_with_constraints.csv', index=False)

print("Dataset saved as 'dataset_truck_orders_cities_with_constraints.csv'")


Dataset saved as 'dataset_truck_orders_cities_with_constraints.csv'


In [4]:
import pandas as pd

df = pd.read_csv('dataset_truck_orders_cities_with_constraints.csv')

route_counts = df.groupby(['City_Origin', 'City_Destination']).size().reset_index(name='Total_Count')

total_routes = route_counts['Total_Count'].sum()

route_counts['Probability'] = route_counts['Total_Count'] / total_routes

print(route_counts[['City_Origin', 'City_Destination', 'Total_Count', 'Probability']])

route_counts.to_csv('any_truck_with_probabilities.csv', index=False)



   City_Origin City_Destination  Total_Count  Probability
0            A                B            5     0.020833
1            A                C            7     0.029167
2            A                D            9     0.037500
3            A                E            7     0.029167
4            A                F            5     0.020833
5            B                A            4     0.016667
6            B                C            7     0.029167
7            B                D           12     0.050000
8            B                E            5     0.020833
9            B                F            5     0.020833
10           C                A            8     0.033333
11           C                B           11     0.045833
12           C                D            6     0.025000
13           C                E            8     0.033333
14           C                F           13     0.054167
15           D                A           11     0.045833
16           D

In [5]:
probability_sum = route_counts['Probability'].sum()

In [6]:
probability_sum

np.float64(1.0)

In [23]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv('dataset_truck_orders_cities_with_constraints.csv')

# Agrupar por Day, City_Origin y City_Destination, y calcular los recuentos totales
route_counts = df.groupby(['Day', 'City_Origin', 'City_Destination']).size().reset_index(name='Total_Count')

# Calcular el total de rutas por cada día
route_counts['Total_Day_Routes'] = route_counts.groupby('Day')['Total_Count'].transform('sum')

# Calcular la probabilidad de cada ruta en relación con su día específico
route_counts['Probability'] = route_counts['Total_Count'] / route_counts['Total_Day_Routes']

# Mostrar el DataFrame final con la columna 'Day'
print(route_counts[['Day', 'City_Origin', 'City_Destination', 'Total_Count', 'Probability']])

# Guardar el DataFrame a un nuevo archivo CSV
route_counts.to_csv('truck_routes_probabilities_by_day.csv', index=False)


           Day City_Origin City_Destination  Total_Count  Probability
0       Friday           A                C            2     0.068966
1       Friday           A                D            1     0.034483
2       Friday           B                C            1     0.034483
3       Friday           B                D            2     0.068966
4       Friday           B                E            2     0.068966
..         ...         ...              ...          ...          ...
145  Wednesday           E                F            1     0.029412
146  Wednesday           F                A            1     0.029412
147  Wednesday           F                C            1     0.029412
148  Wednesday           F                D            1     0.029412
149  Wednesday           F                E            1     0.029412

[150 rows x 5 columns]


In [27]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv('dataset_truck_orders_cities_with_constraints.csv')

# Agrupar por Day y calcular los recuentos totales de rutas para cada día
day_counts = df.groupby('Day').size().reset_index(name='Total_Count')

# Calcular el total de rutas (la suma de todas las rutas en todos los días)
total_routes = day_counts['Total_Count'].sum()

# Calcular la probabilidad de cada día en función del total global de rutas
day_counts['Probability'] = day_counts['Total_Count'] / total_routes

# Mostrar el DataFrame final con la columna 'Day'
print(day_counts[['Day', 'Total_Count', 'Probability']])

# Guardar el DataFrame a un nuevo archivo CSV
day_counts.to_csv('routes_probabilities_by_day.csv', index=False)


         Day  Total_Count  Probability
0     Friday           29     0.120833
1     Monday           39     0.162500
2   Saturday           35     0.145833
3     Sunday           36     0.150000
4   Thursday           29     0.120833
5    Tuesday           38     0.158333
6  Wednesday           34     0.141667


In [31]:
import numpy as np
prob = np.zeros([len(cities),len(cities),len(days_of_week)])

dayidx=-1

for day in days_of_week:
    dayidx+=1
    salidaidx=-1
    destinoidx=-1
    for salida in cities:
         salidaidx+=1
         destinoidx=-1

         print(salidaidx)
         for destino in cities:
              destinoidx+=1
              if salida != destino : 
                bool1=df['City_Origin']==salida
                bool2=df['City_Destination']==destino
                bool3=df['Day']==day

                if df[bool1 & bool2 & bool3].shape[0] ==0:
                    prob[salidaidx,destinoidx,dayidx]=0
                else:
                    prob[salidaidx,destinoidx,dayidx]=df[bool1 & bool2 & bool3].shape[0]/df.shape[0]
prob

0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5
0
1
2
3
4
5


array([[[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ],
        [0.        , 0.0125    , 0.        , 0.00416667, 0.        ,
         0.00416667, 0.        ],
        [0.00416667, 0.        , 0.        , 0.00833333, 0.00833333,
         0.00416667, 0.00416667],
        [0.00416667, 0.00416667, 0.00416667, 0.0125    , 0.00416667,
         0.00416667, 0.00416667],
        [0.00416667, 0.00833333, 0.        , 0.        , 0.        ,
         0.00416667, 0.0125    ],
        [0.00416667, 0.00416667, 0.00416667, 0.        , 0.        ,
         0.00833333, 0.        ]],

       [[0.        , 0.        , 0.00416667, 0.0125    , 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ],
        [0.00416667, 0.00833333, 0.00833333, 0.        , 0.00416667,
         0.00416667, 0.        ],
        [0.00416667, 0.01666667, 0.00416667, 0.        , 0.00833333,
  